In [ ]:
from qdrant_client import QdrantClient
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage

In [ ]:
client = QdrantClient(
    path='../db/qdrant_db',
)

collection_name='books-rec-project3'

In [5]:
with open('../data/stopwords/russian', 'r') as f:
    lines = [line.strip() for line in f.readlines()]

In [7]:
lines[0]

'и'

In [ ]:
client.close()